<a href="https://colab.research.google.com/github/GiuliaTuccio/ScalableML-DL/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab2: sentence BERT
Fine-Tune a pre-trained model


installing all the libraries to implement the lab


In [1]:
!pip install tokenizers
!pip install transformers
!pip install wget
!pip install sentence_transformers


from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample


import gzip
import csv
import wget
import os
import re
import json
import string
import sklearn
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import math
from tensorflow.keras.layers import Input
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
from torch.utils.data import DataLoader



     |████████████████████████████████| 3.3 MB 12.5 MB/s 
     |████████████████████████████████| 3.3 MB 10.8 MB/s 
     |████████████████████████████████| 61 kB 488 kB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=76994cbf190c81173f2ad148cbbb127127e92f6619bc34b8edb425ae37463261
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 27.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=9aa27a5391b764dc354856f33a7cf96953cebfc5a9012f3b79cdc9ce7a8

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Regression task**

Load the data and then build the model required using the sentence transformer library. As loss function define the Mean Squared Error between the label sentence similarity and the predicted one (through the cosine similarity)

In [3]:
train_path = '/content/sts-train.csv'

# will be used for the evaluation phase
df_train = pd.read_csv('/content/sts-train.csv', sep='\t', header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_train.columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']

# will be used for the training phase
train_samples_sts=[]
with open(train_path, newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 5.0  # Normalize score to range 0 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     train_samples_sts.append(inp_example)
     #print(inp_example)


b'Skipping line 5553: expected 7 fields, saw 9\nSkipping line 5554: expected 7 fields, saw 9\nSkipping line 5555: expected 7 fields, saw 9\nSkipping line 5556: expected 7 fields, saw 9\nSkipping line 5557: expected 7 fields, saw 9\nSkipping line 5558: expected 7 fields, saw 9\nSkipping line 5559: expected 7 fields, saw 9\nSkipping line 5560: expected 7 fields, saw 9\nSkipping line 5561: expected 7 fields, saw 9\nSkipping line 5562: expected 7 fields, saw 9\nSkipping line 5563: expected 7 fields, saw 9\nSkipping line 5564: expected 7 fields, saw 9\nSkipping line 5565: expected 7 fields, saw 9\nSkipping line 5566: expected 7 fields, saw 9\nSkipping line 5567: expected 7 fields, saw 9\nSkipping line 5568: expected 7 fields, saw 9\nSkipping line 5569: expected 7 fields, saw 9\nSkipping line 5570: expected 7 fields, saw 9\nSkipping line 5571: expected 7 fields, saw 9\nSkipping line 5572: expected 7 fields, saw 9\nSkipping line 5573: expected 7 fields, saw 9\nSkipping line 5574: expected 7 f

In [4]:
# will be used for the evaluation phase
df_dev = pd.read_csv('/content/sts-dev.csv', sep='\t', header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_dev.columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']

# used for the training
dev_samples_sts=[]

with open('/content/sts-dev.csv', newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 5.0  # Normalize score to range 0 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     dev_samples_sts.append(inp_example)
     #print(inp_example)

b'Skipping line 1479: expected 7 fields, saw 9\nSkipping line 1480: expected 7 fields, saw 9\nSkipping line 1481: expected 7 fields, saw 9\nSkipping line 1482: expected 7 fields, saw 9\nSkipping line 1483: expected 7 fields, saw 9\nSkipping line 1484: expected 7 fields, saw 9\nSkipping line 1485: expected 7 fields, saw 9\nSkipping line 1486: expected 7 fields, saw 9\nSkipping line 1487: expected 7 fields, saw 9\nSkipping line 1488: expected 7 fields, saw 9\nSkipping line 1489: expected 7 fields, saw 9\nSkipping line 1490: expected 7 fields, saw 9\nSkipping line 1491: expected 7 fields, saw 9\nSkipping line 1492: expected 7 fields, saw 9\nSkipping line 1493: expected 7 fields, saw 9\nSkipping line 1494: expected 7 fields, saw 9\nSkipping line 1495: expected 7 fields, saw 9\nSkipping line 1496: expected 7 fields, saw 9\nSkipping line 1497: expected 7 fields, saw 9\nSkipping line 1498: expected 7 fields, saw 9\nSkipping line 1499: expected 7 fields, saw 9\nSkipping line 1500: expected 7 f

In [5]:
test_line = '/content/sts-test.csv'

# will be used for the evaluation phase
df_test = pd.read_csv(test_line, sep='\t', header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_test.columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']


# used for the evaluation phase
test_samples_sts=[]
with open(test_line, newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 5.0  # Normalize score to range 0 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     test_samples_sts.append(inp_example)
     #print(inp_example)

b'Skipping line 626: expected 7 fields, saw 9\nSkipping line 627: expected 7 fields, saw 9\nSkipping line 628: expected 7 fields, saw 9\nSkipping line 629: expected 7 fields, saw 9\nSkipping line 630: expected 7 fields, saw 9\nSkipping line 631: expected 7 fields, saw 9\nSkipping line 632: expected 7 fields, saw 9\nSkipping line 633: expected 7 fields, saw 9\nSkipping line 634: expected 7 fields, saw 9\nSkipping line 635: expected 7 fields, saw 9\nSkipping line 636: expected 7 fields, saw 9\nSkipping line 637: expected 7 fields, saw 9\nSkipping line 638: expected 7 fields, saw 9\nSkipping line 639: expected 7 fields, saw 9\nSkipping line 640: expected 7 fields, saw 9\nSkipping line 641: expected 7 fields, saw 9\nSkipping line 642: expected 7 fields, saw 9\nSkipping line 643: expected 7 fields, saw 9\nSkipping line 644: expected 7 fields, saw 9\nSkipping line 645: expected 7 fields, saw 9\nSkipping line 646: expected 7 fields, saw 9\nSkipping line 647: expected 7 fields, saw 9\nSkipping

Define the pre-trained model as bert-base-uncased and using sentence-transformer library to define the word-embedding

In [6]:
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Define the parameters as required by the lab assignement

In [7]:
train_batch_size = 16
learn_rate = 2e-5
num_epochs = 1

Add to the pre-trained model the mean pooling layer and modify the model by adding the pooling layer

In [8]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)


model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [9]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

Define the DataLoader, as usually in Python, to load the training data
Compute the cosine similarity between the two sentence embeddings and use Mean Squared erro as loss function (defined as default by Cosine Similarity Loss)

CosineSimilarityLoss expects, it minimizes the following loss: ||input_label - cos_score_transformation(cosine_sim(u,v))||_2.
    


In [10]:
train_dataloader = DataLoader(train_samples_sts, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model) #loss function=MSE

Define the evaluator which evaluates the Person similarity by predicting the label of two sentence ambeddings and compute the pearsorn coefficient between the predicted and the true label

In [11]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples_sts)

In [12]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up

In [13]:
# Save path to store our model
regression_path = '/sts-regression'
model_save_path = regression_path

Fit the model using train data, loss function and evaluator over the developement dataset.

In [14]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          save_best_model=model_save_path)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

**Define the model as the one trained and evaluate it with respect to cosine similarity**

\
For the evaluation part we use both the sentence-transformer library using EmbeddingSimilarity evaluator, as well as implementing it ourselves.
In the latter, we first encode the test sentences using the trained model and secondly compute the cosine similarity as 


**cosine sim = 1 - cosine distance between the sentence embedding**

In [15]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples_sts)
test_evaluator(model, output_path=model_save_path)

0.8090364444454816

In [16]:
embeddings1_sts = model.encode(df_test['sentenceA'], batch_size=16, convert_to_numpy=True)
embeddings2_sts = model.encode(df_test['sentenceB'], batch_size=16, convert_to_numpy=True)

In [17]:
# Define cosine Similarity as 1 - cosine distance
# smallest distance means higher similarity.
score_pred_sim_sts = 1 - (sklearn.metrics.pairwise.paired_cosine_distances(embeddings1_sts, embeddings2_sts))
print(score_pred_sim_sts)

corr_test_sim_sts = scipy.stats.spearmanr(score_pred_sim_sts, df_test['value'])[0]
corr_test_sim_sts

[0.7749426  0.81996965 0.9201771  ... 0.86536455 0.9336237  0.9200722 ]


0.8308473645209083

**Classification Task**

Load the data from the website and unzip it. 
Then use the same model as before, changing the loss function to the softmax between the the sentence embeddings and the difference between them.



In [18]:
print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nlp.stanford.edu/projects/snli/snli_1.0.zip'

# Download the file (if we haven't already)
if not os.path.exists('./snli_1.0.zip'):
    wget.download(url, './snli_1.0.zip')

In [19]:
!unzip snli_1.0.zip

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_test.txt  
  inflating: snli_1.0/snli_1.0_train.jsonl  
  inflating: snli_1.0/snli_1.0_train.txt  
  inflating: __MACOSX/._snli_1.0     


In [20]:
train_path = '/content/snli_1.0/snli_1.0_train.jsonl'

df_class = pd.read_json("snli_1.0/snli_1.0_train.jsonl", lines=True)
df_class


,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
...,...,...,...,...,...,...,...,...,...,...
550147,[contradiction],2267923837.jpg#3,contradiction,2267923837.jpg#3r1c,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,four kids won awards for 'cleanest feet',( ( four kids ) ( ( won awards ) ( ( ( for ` )...,(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD wo...
550148,[neutral],2267923837.jpg#3,neutral,2267923837.jpg#3r1n,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,"four homeless children had their shoes stolen,...",( ( ( ( ( ( four ( homeless children ) ) ( had...,(ROOT (S (S (NP (CD four) (JJ homeless) (NNS c...
550149,[neutral],7979219683.jpg#2,neutral,7979219683.jpg#2r1n,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a bodysuit is competing in a surfing ...,( ( ( A man ) ( in ( a bodysuit ) ) ) ( ( is (...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...
550150,[contradiction],7979219683.jpg#2,contradiction,7979219683.jpg#2r1c,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a business suit is heading to a board...,( ( ( A man ) ( in ( a ( business suit ) ) ) )...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...


Perform explanatory analysis

In [21]:
print(df_class['gold_label'])

0               neutral
1         contradiction
2            entailment
3               neutral
4            entailment
              ...      
550147    contradiction
550148          neutral
550149          neutral
550150    contradiction
550151       entailment
Name: gold_label, Length: 550152, dtype: object


In [22]:
df_class.gold_label.value_counts()

entailment       183416
contradiction    183187
neutral          182764
-                   785
Name: gold_label, dtype: int64

In [23]:
print(len(df_class.gold_label.index))

550152


In [24]:
# train_class = []

# delete=[]

# for i in df_class.index:
#   print(i)
  
#   if df_class['gold_label'][i] == 'contradiction':
#     label_id = 0
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   elif df_class['gold_label'][i] == 'entailment':
#     label_id = 1
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   elif df_class['gold_label'][i] == 'neutral':
#     label_id = 2
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   else:
#     delete_id = 3
#     inp_sample =  InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=delete_id)
#     delete.append(inp_sample)
#     #print('..................................................................................................................')
    

Assign to phrases classified as contradiction label 0 ,
                                entailment label 1,
                                neutral label 2 

In [25]:
train_class = []

for i in df_class.index:
  #print(i)
  
  if df_class['gold_label'][i] == 'contradiction':
    label_id = 0
    
  elif df_class['gold_label'][i] == 'entailment':
    label_id = 1
    
  elif df_class['gold_label'][i] == 'neutral':
    label_id = 2
  
  inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
  #print(inp_sample)
  train_class.append(inp_sample)

In [28]:
dev_path = 'snli_1.0/snli_1.0_dev.jsonl'

df_class_dev = pd.read_json(dev_path, lines=True)

dev_class = []

for i in df_class_dev.index:
  if df_class_dev['gold_label'][i] == 'contradiction':
    label_id = 0
  elif df_class_dev['gold_label'][i] == 'entailment':
    label_id  = 1
  elif df_class_dev['gold_label'][i] == 'neutral':
    label_id = 2
  inp_sample = InputExample(texts=[df_class_dev['sentence1'][i], df_class_dev['sentence2'][i]], label=label_id)
  #print(inp_sample)
  dev_class.append(inp_sample)

In [29]:
test_path = 'snli_1.0/snli_1.0_test.jsonl'
df_class_test = pd.read_json(test_path, lines=True)

test_class=[]
label =[]

for i in df_class_test.index:
  if df_class_test['gold_label'][i] == 'contradiction':
    label_id = 0
  elif df_class_test['gold_label'][i] == 'entailment':
    label_id = 1
  elif df_class_test['gold_label'][i] == 'neutral':
    label_id = 2
  inp_sample = InputExample(texts=[df_class_test['sentence1'][i], df_class_test['sentence2'][i]], label=label_id)
  label.append(label_id)
  #print(inp_sample)
  test_class.append(inp_sample)

#print(label)

In [30]:
# Read the dataset
train_batch_size = 16

# Define the path to save the model
model_save_path_class = 'content/training_nli_'

Define the model, which is the same as before, changing the dataset

In [31]:
train_dataloader = DataLoader(train_class, shuffle=True, batch_size=train_batch_size)

In [32]:
#creating custom model, by using mean pooling of the word embeddings given as input
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [34]:
#     SentenceTransformer model : BERT base
#     Dimension of the sentence embeddings
#     Number of different labels : 3 (0,1,2)
#     Concatenate vectors u,v for the softmax classifier? T
#     Add abs(u-v) for the softmax classifier? T
#     Add u*v for the softmax classifier? F
#     If not set, uses nn.CrossEntropyLoss() T

train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [35]:
# Define the evaluator on the development samples 
# dev_samples_sts
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_class, batch_size=train_batch_size, name='snli-dev')

In [36]:
# Configure the training
num_epochs = 1

# Define warm up steps
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path_class
          )


Evaluate using bothe the library in sentence-transformer and using the cosine similarity

In [37]:
#model = SentenceTransformer(model_save_path_class)
class_model_path = '/content/drive/MyDrive/Colab Notebooks/training_nli_'
model = SentenceTransformer(class_model_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_class, name='snli-test')
test_evaluator(model, output_path=class_model_path)

0.2471157203244841

In [38]:
embeddings1_nli = model.encode(df_class_test['sentence1'], batch_size=16, convert_to_numpy=True)
embeddings2_nli = model.encode(df_class_test['sentence2'], batch_size=16, convert_to_numpy=True)


In [39]:
cosine_scores_nli = 1 - (sklearn.metrics.pairwise.paired_cosine_distances(embeddings1_nli, embeddings2_nli))
eval_pearson_cosine_nli = scipy.stats.spearmanr(label, cosine_scores_nli)[0] 

print(eval_pearson_cosine_nli)

0.23967547665587127


Using the previuos model, fine-tuned on sts-dataset and evaluate on the sts dataset

In [40]:
train_dataloader = DataLoader(train_class[0:100000], shuffle=True, batch_size=train_batch_size)

In [41]:
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples_sts, batch_size=train_batch_size, name='sts-dev')

In [42]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path_class
          )


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6250 [00:00<?, ?it/s]

In [43]:
model_best = SentenceTransformer(model_save_path_class)
#model_best = SentenceTransformer(class_model_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples_sts, name='sts-test')
test_evaluator(model_best, output_path=class_model_path)

0.7421078419699247

In [44]:
embeddings1 = model_best.encode(df_test['sentenceA'], batch_size=16, convert_to_numpy=True)
embeddings2 = model_best.encode(df_test['sentenceB'], batch_size=16, convert_to_numpy=True)
cosine_scores = 1 - (sklearn.metrics.pairwise.paired_cosine_distances(embeddings1, embeddings2))
eval_pearson_cosine = scipy.stats.spearmanr(df_test['value'], cosine_scores)[0] 

print(eval_pearson_cosine)

0.7588174516176172


In [ ]:
#https://www.kaggle.com/rmisra/news-category-dataset/download

In [47]:
print('Downloading dataset...')

# define the path for the news data set and read it
semantic_search_path = '/content/News_Category_Dataset_v2.json'
news = pd.read_json(semantic_search_path, lines=True)

In [48]:
# print the data set to understand how it is presented 
news

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [49]:
# prepare the data used for the implementation of the semantic research
# data is a list composed by both the short description and the headlines and link [headlines, short_description, link]

data = []

for indx in news.index:
  data.append([news['headline'][indx], news['short_description'][indx], news['link'][indx]])
  

In [50]:
# We use the best model to encode all passages, so that we can use it with sematic search

encoder = SentenceTransformer(class_model_path)

In [51]:
# encode the data using the best model
corpus_embeddings = encoder.encode(data, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/6277 [00:00<?, ?it/s]

In [52]:
# Number of data we want to provide as output
top_k = 8

# ask for an input research
input = input("Please enter the research: ")

Please enter the research: house burning


In [53]:
# encode the research query using the best_model
query_embedding = encoder.encode(input, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
print(top_results[0])

print("Input:", input)
print("\nTop {} most similar sentences in the news data base:".format(top_k))

for score, idx in zip(top_results[0], top_results[1]):
  print(data[idx],"\n", "(Score: {:.3f})".format(score))
  print("\n------------------------\n")

tensor([0.6983, 0.6187, 0.5883, 0.5533, 0.5393, 0.5346, 0.5081, 0.5074],
       device='cuda:0')




Input: house burning

Top 8 most similar sentences in the news data base:
['House Fire Kills 6', '', 'https://www.huffingtonpost.com/entry/north-carolina-house-fire_n_5741178.html'] 
 (Score: 0.698)


['House GOP Crackdown Continues', '', 'https://www.huffingtonpost.com/entry/jason-chaffetz-strips-mea_n_7629404.html'] 
 (Score: 0.619)


['Burning Buildings in Baltimore', '', 'https://www.huffingtonpost.com/entry/burning-buildings-in-balt_b_7182782.html'] 
 (Score: 0.588)


['Pita Pit Fires Employees Over Facebook', 'Ouch.', 'https://www.huffingtonpost.com/entry/pita-pit-facebook_us_5b9dd237e4b03a1dcc8d5008'] 
 (Score: 0.553)


['Is Your House Haunted? Or Just Dirty?', '', 'https://www.huffingtonpost.com/entry/ghosts-caused-mold-clarkson-university_n_7006836.html'] 
 (Score: 0.539)


['Planned Parenthood Fire In Southern California Was Arson, Authorities Say', 'Cameras recorded someone t